In [1]:
import pandas as pd
import matplotlib .pyplot as plt
from persiantools.jdatetime import JalaliDate

order = pd.read_csv('orders.csv')
product = pd.read_excel('product.xlsx')
history = pd.read_csv('tarikhche kharid.csv')  


merge_order_product = pd.merge(order, product, left_on='ID_Item', right_on='id', how='inner')
merge_order_product['total_sales'] = merge_order_product['Amount_Gross_Order'] * merge_order_product['Quantity_item']



grouped = merge_order_product.groupby(['city_name_fa', 'product_title_fa']).agg(
    total_sales=('total_sales', 'sum'),
    total_quantity=('Quantity_item', 'sum')
).reset_index()
# print(grouped)


city_sales = grouped.groupby('city_name_fa')['total_sales'].sum().reset_index()
# print(city_sales)


top_cities = city_sales.nlargest(3, 'total_sales')['city_name_fa']

# print(top_cities)


top_products = grouped[grouped['city_name_fa'].isin(top_cities)].groupby('city_name_fa').apply(
    lambda x: x.nlargest(3, 'total_sales')
).reset_index(drop=True)


short_products = grouped[grouped['city_name_fa'].isin(top_cities)].groupby('city_name_fa').apply(
    lambda x: x.nsmallest(3, 'total_sales')
).reset_index(drop=True)

print("Top 3 Best-Selling Products in the Top 3 Cities:")
print(top_products[['city_name_fa', 'product_title_fa', 'total_sales', 'total_quantity']])

print("\nTop 3 Short-Selling Products in the Top 3 Cities:")
print(short_products[['city_name_fa', 'product_title_fa', 'total_sales', 'total_quantity']])



most_profitable_city = city_sales.loc[city_sales['total_sales'].idxmax()]

lowest_income_city = city_sales.loc[city_sales['total_sales'].idxmin()]
print("Most Profitable City:")
print(f"City: {most_profitable_city['city_name_fa']}, Total Sales: {most_profitable_city['total_sales']}")
print("\nLowest Income City:")
print(f"City: {lowest_income_city['city_name_fa']}, Total Sales: {lowest_income_city['total_sales']}")



order['DateTime_CartFinalize'] = pd.to_datetime(order['DateTime_CartFinalize'])

order['date_only'] = order['DateTime_CartFinalize'].dt.date


def to_jalali_date(gregorian_date):
    jalali = JalaliDate.to_jalali(gregorian_date.year, gregorian_date.month, gregorian_date.day)
    return jalali


order['jalali_date'] = order['date_only'].apply(to_jalali_date)

order['jalali_month'] = order['jalali_date'].apply(lambda d: d.month)


def jalali_season(month):
    if month in [1, 2, 3]:
        return 'Spring'
    elif month in [4, 5, 6]:
        return 'Summer'
    elif month in [7, 8, 9]:
        return 'Autumn'
    else:
        return 'Winter'

order['season'] = order['jalali_month'].apply(jalali_season)


season_counts = order['season'].value_counts().reindex(['Spring', 'Summer', 'Autumn', 'Winter'])


plt.figure(figsize=(8,5))
season_counts.plot(color='purple')
plt.title('Order Counts by Shamsi Season')
plt.xlabel('Season')
plt.ylabel('Number of Orders')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'orders.csv'